In [1]:
#pip install imblearn

In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import time
import operator
from collections import Counter
from imblearn.combine import SMOTEENN
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
# Create DataFrame from CSV
t0 = time.time()
filtered_lyric_TF_df = pd.read_csv('../../Data/filtered_lyric_TF.csv')
filtered_lyric_TF_df = filtered_lyric_TF_df.drop([0,1])
filtered_lyric_TF_df.index = filtered_lyric_TF_df.index - 2
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df.head()

Run time: 42.24658513069153 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,professed,plottin,sideline,sufficient,girly,reek,duffel,bitter,staff,eighth
0,willow,Taylor Swift,pop,8.0,"['dance', 'pop']",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"['indie', 'pop', 'electropop']",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,WITHOUT YOU,The Kid LAROI,pop,8.0,['australian'],0.662,0.413,0.0,1.0,0.0299,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Heat Waves,Glass Animals,pop,8.0,"['shiver', 'indietronica', 'gauze']",0.761,0.525,11.0,1.0,0.0944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"['dance', 'pop', 'electropop', 'post-teen']",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
# Create a dictionary of category names and ids
cat_name_id = {}
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    cat_id = list(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat]['category_id'])[0]
    cat_name_id[cat] = cat_id
    print(cat, cat_id)

blues 0.0
classical 1.0
country 2.0
funk 3.0
hiphop 4.0
indie_alt 5.0
jazz 6.0
metal 7.0
pop 8.0
punk 9.0
rnb 10.0
rock 11.0
romance 12.0
soul 13.0


In [4]:
# Create a genre lists column
t0 = time.time()
genre_lists = []
for index, row in filtered_lyric_TF_df.iterrows():
    genres = row['genre_list'].replace('[', '').replace(']', '').replace("'", '').split(', ')
    genre_lists.append(genres)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
len(genre_lists)

Run time: 47.56153392791748 seconds


8041

In [5]:
# Replace the genre_list column
filtered_lyric_TF_df['genre_list'] = genre_lists

In [6]:
# Print how many songs there are in each category
for cat in category_list:
    song_count = len(filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 424 songs in the blues category.
There are 95 songs in the classical category.
There are 1257 songs in the country category.
There are 282 songs in the funk category.
There are 683 songs in the hiphop category.
There are 660 songs in the indie_alt category.
There are 198 songs in the jazz category.
There are 898 songs in the metal category.
There are 693 songs in the pop category.
There are 569 songs in the punk category.
There are 362 songs in the rnb category.
There are 1229 songs in the rock category.
There are 280 songs in the romance category.
There are 411 songs in the soul category.


In [7]:
# Get the most popular genres for each category
t0 = time.time()
category_genres = {}
for category in category_list:
    genres = []
    for index, row in filtered_lyric_TF_df[filtered_lyric_TF_df['category_name']==category].iterrows():
        genres.extend(row['genre_list'])
    unique_genres = list(set(genres))
    genre_counts = []
    for genre in unique_genres:
        genre_counts.append((genre, genres.count(genre)))
    category_genres[category] = sorted(genre_counts, key = lambda x: x[1], reverse=True)[0:5]
    print(category, category_genres[category])
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

blues [('blues', 260), ('electric', 190), ('modern', 168), ('traditional', 146), ('classic', 106)]
classical [('classical', 17), ('operatic', 14), ('dance', 12), ('', 9), ('pop', 9)]
country [('country', 977), ('contemporary', 847), ('modern', 277), ('pop', 88), ('nashville', 78)]
funk [('funk', 133), ('soul', 118), ('motown', 97), ('quiet', 82), ('disco', 79)]
hiphop [('rap', 346), ('hip', 305), ('pop', 228), ('trap', 192), ('southern', 143)]
indie_alt [('indie', 291), ('alternative', 181), ('new', 155), ('modern', 148), ('art', 144)]
jazz [('vocal', 93), ('contemporary', 83), ('jazz', 79), ('adult', 64), ('neo', 38)]
metal [('metal', 368), ('alternative', 286), ('nu', 222), ('rock', 221), ('melodic', 207)]
pop [('pop', 344), ('indie', 180), ('dance', 164), ('post-teen', 129), ('electropop', 129)]
punk [('pop', 195), ('punk', 138), ('skate', 94), ('alternative', 84), ('melodic', 72)]
rnb [('pop', 219), ('r&b', 213), ('urban', 200), ('dance', 169), ('hip', 136)]
rock [('rock', 638), ('

In [8]:
# Find indeces to keep and indeces to remove
t0 = time.time()
cat_genre_dict = {'pop': ['pop'], 
                  'hiphop': ['rap', 'hip', 'trap'],
                  'metal': ['metal'], 
                  'rock': ['classic'], 
                  'jazz': ['jazz'], 
                  'rnb': ['r&b'], 
                  'romance': ['soft'], 
                  'soul': ['soul'], 
                  'indie_alt': ['indie', 'alternative'],
                  'classical': ['classical', 'operatic', 'dance'], 
                  'blues': ['modern', 'blues'],
                  'funk': ['funk', 'disco'], 
                  'punk': ['punk'],
                  'country': ['country']} 
indeces_to_keep = []
indeces_to_remove = []
for index, row in filtered_lyric_TF_df.iterrows():
    keep = False
    category = row['category_name']
    for genre in cat_genre_dict[category]:
        if genre in row['genre_list']:
            keep = True
            break
    if keep == True:
        indeces_to_keep.append(index)
    else:
        indeces_to_remove.append(index)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
print(len(indeces_to_keep))
len(indeces_to_remove)

Run time: 39.143298864364624 seconds
3990


4051

In [26]:
# Create non destructive copy and remove the unnecessary indeces
t0 = time.time()
filtered_lyric_TF_df2 = filtered_lyric_TF_df.drop(indeces_to_remove)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')
filtered_lyric_TF_df2.head(3)

Run time: 0.40039491653442383 seconds


,song_name,artist_name,category_name,category_id,genre_list,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,...,professed,plottin,sideline,sufficient,girly,reek,duffel,bitter,staff,eighth
0,willow,Taylor Swift,pop,8.0,"[dance, pop]",0.392,0.574,7.0,1.0,0.1700,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Stay Next To Me (with Chelsea Cutler),Quinn XCII,pop,8.0,"[indie, pop, electropop]",0.581,0.584,2.0,1.0,0.2840,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,r u ok,Tate McRae,pop,8.0,"[dance, pop, electropop, post-teen]",0.666,0.593,2.0,1.0,0.0373,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
# Remove the romance, punk, and blues categories
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='romance']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='punk']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='blues']
filtered_lyric_TF_df2 = filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']!='indie_alt']
len(filtered_lyric_TF_df2)

3082

In [28]:
# Print how many songs there are in each category 
category_list = ['blues', 'classical', 'country', 'funk', 'hiphop', 'indie_alt', 'jazz', 
                 'metal', 'pop', 'punk', 'rnb', 'rock', 'romance', 'soul']
for cat in category_list:
    song_count = len(filtered_lyric_TF_df2[filtered_lyric_TF_df2['category_name']==cat])
    print(f'There are {song_count} songs in the {cat} category.')

There are 0 songs in the blues category.
There are 38 songs in the classical category.
There are 977 songs in the country category.
There are 163 songs in the funk category.
There are 447 songs in the hiphop category.
There are 0 songs in the indie_alt category.
There are 79 songs in the jazz category.
There are 368 songs in the metal category.
There are 344 songs in the pop category.
There are 0 songs in the punk category.
There are 213 songs in the rnb category.
There are 245 songs in the rock category.
There are 0 songs in the romance category.
There are 208 songs in the soul category.


In [29]:
# Define the features set
X = filtered_lyric_TF_df2.copy()
X = X.drop(['song_name', 'artist_name', 'category_name', 'category_id', 'genre_list'], axis=1)
X.head(3)

,audio_ft_danceability,audio_ft_energy,audio_ft_key,audio_ft_mode,audio_ft_speechiness,audio_ft_acousticness,audio_ft_instrumentalness,audio_ft_liveness,audio_ft_valence,audio_ft_tempo,...,professed,plottin,sideline,sufficient,girly,reek,duffel,bitter,staff,eighth
0,0.392,0.574,7.0,1.0,0.1700,0.8330,0.00179,0.145,0.529,81.112,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.581,0.584,2.0,1.0,0.2840,0.0805,0.00000,0.366,0.756,179.954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.666,0.593,2.0,1.0,0.0373,0.3180,0.00000,0.414,0.329,140.013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [30]:
# Define the target set
y = filtered_lyric_TF_df2['category_id'].ravel()
y[:5]

array([8., 8., 8., 8., 8.])

In [31]:
# Split into Train and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2311, 12081)
(771, 12081)
(2311,)
(771,)


# Combination Sampling with SMOTEENN

In [32]:
# Perform combination sampling on the data
t0 = time.time()
smote_enn = SMOTEENN(random_state=0)
X_resampled, y_resampled = smote_enn.fit_resample(X, y)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 49.84922695159912 seconds


In [33]:
# Count the resampled classes
Counter(y_resampled)

Counter({1.0: 168,
         2.0: 17,
         3.0: 98,
         4.0: 85,
         6.0: 65,
         7.0: 65,
         8.0: 103,
         10.0: 120,
         11.0: 93,
         13.0: 116})

# Fitting the Model

In [34]:
# Create a random forest classifier
rf_model = RandomForestClassifier(n_estimators=500, random_state=78)

In [35]:
# Fit the model
t0 = time.time()
rf_model = rf_model.fit(X_resampled, y_resampled)
t1 = time.time()
print(f'Run time: {t1-t0} seconds')

Run time: 6.563177108764648 seconds


# Making Predictions

In [36]:
# Make predictions using the testing data
predictions = rf_model.predict(X_test)
predictions

array([ 8.,  7., 10., 10.,  8.,  8., 11., 11., 10., 11.,  3.,  8.,  4.,
        4., 13.,  8., 13.,  8.,  8.,  7.,  6., 10.,  7.,  4., 13.,  3.,
        7.,  8., 11., 11.,  8.,  4., 11., 11.,  8.,  8.,  6., 13.,  3.,
        8., 11., 10.,  8., 11.,  8., 13.,  4., 13.,  7., 10., 13.,  8.,
        7., 13., 13., 13.,  4.,  8.,  8.,  7.,  8., 13.,  8.,  3., 10.,
        3.,  6.,  7., 13.,  4.,  7., 11., 13., 10.,  4.,  3.,  7.,  8.,
       11.,  8., 13., 13.,  8.,  8., 13.,  7.,  7.,  4.,  8.,  4.,  4.,
       11., 10.,  3., 13.,  4., 13.,  8.,  7.,  7.,  4.,  3., 13., 11.,
        8., 11., 13.,  8., 10., 13.,  6.,  8., 13.,  4.,  8., 10.,  8.,
        8., 10., 13.,  8.,  8., 11.,  3.,  8., 13.,  8.,  4.,  4.,  4.,
        7., 13.,  8., 11., 13.,  4.,  8.,  4., 11.,  3.,  8.,  4.,  4.,
       13.,  3.,  4.,  3.,  8., 13., 13.,  8.,  8.,  8.,  4., 11.,  7.,
        8.,  8., 13.,  3.,  8., 11.,  3.,  7.,  8.,  8.,  4., 13., 13.,
       13.,  8., 11.,  7., 11.,  6., 11., 10.,  8.,  7.,  4., 11

# Model Evaluation

In [37]:
# Calculate the accuracy score
acc_score = accuracy_score(y_test, predictions)
print(f'Accuracy Score: {acc_score}')

Accuracy Score: 0.5110246433203631


In [ ]:
Accuracy Score: 0.5063583815028901


In [38]:
# Calculate the feature importance in the Random Forest Model
importances = rf_model.feature_importances_
importances

array([0.01038886, 0.00983012, 0.00309099, ..., 0.00011426, 0.        ,
       0.        ])

In [39]:
# Sort the features by importance
sorted_features = sorted(zip(importances, X.columns), reverse=True)
sorted_features

[(0.011847560672588917, 'audio_ft_acousticness'),
 (0.01150375932184472, 'audio_ft_duration_ms'),
 (0.010687847697521343, 'audio_ft_valence'),
 (0.010388855789959338, 'audio_ft_danceability'),
 (0.009830115224605354, 'audio_ft_energy'),
 (0.007902889441578551, 'audio_ft_instrumentalness'),
 (0.006870007433861627, 'audio_ft_speechiness'),
 (0.0060407691878310915, 'oh'),
 (0.005367967509180752, 'audio_ft_tempo'),
 (0.004825656540916004, 'baby'),
 (0.004478091280358986, 'dont'),
 (0.004417329508386296, 'know'),
 (0.004361028051458159, 'love'),
 (0.004336982900015771, 'youre'),
 (0.0043293320179766245, 'im'),
 (0.004201450126405609, 'girl'),
 (0.0041721564228186575, 'got'),
 (0.004119441670297987, 'aint'),
 (0.004033546222779123, 'get'),
 (0.004006581327131977, 'niggas'),
 (0.0038336999584900108, 'ooh'),
 (0.003519416518672038, 'shit'),
 (0.0035029091891023547, 'nigga'),
 (0.0035006323884257855, 'gon'),
 (0.0034695524180264727, 'yeah'),
 (0.0034691354223570156, 'one'),
 (0.0034371202855511

# Dropping Lower Ranked Features

In [40]:
# Create a dictionary of features and accuracy scores
features_accuracy = {}
feature_percents = [0.1, 0.2, 0.3]
for percent in feature_percents:
    # Define the features and target sets
    important_features = []
    sum = 0
    count = 0
    while sum < percent:
        sum = sum + sorted_features[count][0]
        important_features.append(sorted_features[count][1])
        count+=1
    X = filtered_lyric_TF_df2[important_features]
    y = filtered_lyric_TF_df2['category_id'].ravel()
    # Split into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)
    # Scale the data
    scaler = StandardScaler()
    X_scaler = scaler.fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    # Fit the random forest model
    rf_model = RandomForestClassifier(n_estimators=500, random_state=78)
    t0 = time.time()
    rf_model = rf_model.fit(X_train_scaled, y_train)
    t1 = time.time()
    print(f'Run time to fit model with {percent*100}% of total features: {t1-t0} seconds')
    # Make predictions
    predictions = rf_model.predict(X_test_scaled)
    acc_score = accuracy_score(y_test, predictions)
    features_accuracy[percent] = acc_score

Run time to fit model with 10.0% of total features: 3.038120985031128 seconds
Run time to fit model with 20.0% of total features: 3.756054162979126 seconds
Run time to fit model with 30.0% of total features: 4.101999044418335 seconds
Run time to fit model with 40.0% of total features: 3.430006980895996 seconds


In [41]:
# Print feature percents and accuracy scores
for percent in features_accuracy:
    acc_score = features_accuracy[percent]
    print(f'Accuracy Score for {percent*100}% of total features: {acc_score}')

Accuracy Score for 10.0% of total features: 0.6407263294422828
Accuracy Score for 20.0% of total features: 0.6900129701686122
Accuracy Score for 30.0% of total features: 0.6848249027237354
Accuracy Score for 40.0% of total features: 0.6796368352788587


In [42]:
# Get the highest accuracy score
best_percent = max(features_accuracy, key=features_accuracy.get)
best_acc = features_accuracy[best_percent]
print(f'The highest accuracy score of {best_acc} was achieved using {best_percent*100}% of all features.')

The highest accuracy score of 0.6900129701686122 was achieved using 20.0% of all features.


# Accuracy score for SMOTEENN
- The highest accuracy score of 0.6434378629500581 was achieved using 20.0% of all features.

# Accuracy scores with different data
- Original <br>
The highest accuracy score of 0.5296296296296297 was achieved using 30.0% of all features.
- Remove romance category <br>
The highest accuracy score of 0.5805843543826579 was achieved using 30.0% of all features.
- Remove punksteam genre from punk category <br>
The highest accuracy score of 0.5872865275142315 was achieved using 40.0% of all features.
<br> [0.2 - 0.5] <br>
- Remove electropop genre from pop category <br>
The highest accuracy score of 0.5916030534351145 was achieved using 20.0% of all features. 
- Remove motown genre from soul category <br>
The highest accuracy score of 0.6032660902977905 was achieved using 40.0% of all features.
- Add dance genre to classical category <br>
The highest accuracy score of 0.6120689655172413 was achieved using 20.0% of all features.
<br> [0.1 - 0.5] <br>
- Add trap genre to hiphop category <br>
The highest accuracy score of 0.6213408876298395 was achieved using 30.0% of all features.
- Add nashville genre to country category <br>
The highest accuracy score of 0.6053604436229205 was achieved using 40.0% of all features. ??
- Replace rock genre with classic <br>
The highest accuracy score of 0.639344262295082 was achieved using 20.0% of all features.
- Replace blues genre with classic <br>
The highest accuracy score of 0.650319829424307 was achieved using 20.0% of all features.
<br> [0.1 - 0.4] <br>
- Replace blues genre with modern <br>
The highest accuracy score of 0.6509433962264151 was achieved using 30.0% of all features.

<br>
The highest accuracy score of 0.6119402985074627 was achieved using 40.0% of all features.
<br>
- Remove punk category <br>
The highest accuracy score of 0.6245847176079734 was achieved using 20.0% of all features. <br>
- Add blues genre <br>
The highest accuracy score of 0.6446808510638298 was achieved using 20.0% of all features. <br>
- Remove blues category <br>
The highest accuracy score of 0.6515679442508711 was achieved using 20.0% of all features. <br>
- Remove indie category <br>
The highest accuracy score of 0.7044270833333334 was achieved using 30.0% of all features. <br>

